In [ ]:
!nvidia-smi

Tue Feb  1 09:35:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install mediapipe
from tensorflow.keras.models import load_model
from cv2 import VideoCapture, waitKey, imshow, cvtColor, flip, COLOR_BGR2RGB, COLOR_RGB2BGR, putText, FONT_HERSHEY_SIMPLEX, resize, rectangle, line
import mediapipe as mp
import cv2
from numpy import asscalar, ndarray, array, expand_dims
from sklearn.metrics import confusion_matrix
import pandas as pd
from pathlib import Path
from csv import writer
from numpy import ndarray, array, zeros, concatenate
from shutil import move

     |████████████████████████████████| 32.7 MB 153 kB/s 


In [ ]:
class Writer:
    temp_path = ''

    def write_to_csv(self, result: ndarray, out_csv: str):
        out_csv = Path(out_csv)

        with out_csv.open('a+') as csv:
            csv_writer = writer(csv)
            csv_writer.writerow(result)

    def write_as_video(self, path, image):
        if self.temp_path != path: 
            self.temp_path = path
            self.writer = VideoWriter(path, VideoWriter_fourcc('M','J','P','G'), 10, (640, 480))

        self.writer.write(image)

In [ ]:
class ImageHandler:
    def draw_results(self, image, results):
        image.flags.writeable = True

        # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        return image


In [ ]:
class MediapipeExtractor:
    
    def extract_landmarks(self, results) -> ndarray:
        result_landmarks = {
            'face': {
                'landmark': results.face_landmarks,
                'shape': (468,)
            },
            'left hand': {
                'landmark': results.left_hand_landmarks,
                'shape': (21,)
            },
            'right hand': {
                'landmark': results.right_hand_landmarks,
                'shape': (21,)
            },
            'pose': {
                'landmark': results.pose_landmarks,
                'shape': (33,)
            }
        }

        result_all = []
        result_temp = []
        for key, result in result_landmarks.items():
            if result['landmark']:
                result_temp = array([array([l.x, l.y, l.z]) for l in result['landmark'].landmark])
                result_temp = result_temp.flatten()
            else:
                result_temp = zeros((result['shape'][0] * 3, ))

            assert(result_temp.shape == (result['shape'][0] * 3, ))
            result_all.append(result_temp)
            
        shape = 0
        result_final = array([])
        for result in result_all:
            shape += result.shape[0]
            result_final = concatenate((result_final, result), axis=0)

        assert(result_final.shape == (shape,))

        return result_final

In [ ]:

from cv2 import VideoWriter_fourcc, VideoWriter

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

me = MediapipeExtractor()
im = ImageHandler()
w = Writer()

source_dir = "/content/drive/MyDrive/test_data/augmented"
source_directory = Path(source_dir)

klasses = ['Address', 'Movie', 'Name', 'Phone', 'Play', 'Please','Work','Your']

classes = list(x.name for x in source_directory.glob('*'))
print(classes)

all_source_ds = []      #source video location list
for category in classes:
    cat_dir = list((source_directory/category).glob("*"))
    all_source_ds.append(cat_dir)
# print(len(all_source_ds))  #GET READY FOR FLOOOD
counter_class = len(classes)
print(counter_class)  





['Movie', 'Name', 'Your', 'Address', 'Play', 'Work', 'Please', 'phone']
8


In [ ]:
final_pred_list = []
final_list = []
final_final_pred_list = []
final_final_list = []
model = '/content/drive/MyDrive/Model_new/ADB_9_95'
model = load_model(str(model))
QR = []         #list of frames

In [ ]:
for i in range(len(classes)):
    category = classes[i]
    current_ds = all_source_ds[i]   #source video location of i'th class
    print(len(current_ds))
    for j in range(len(current_ds[:100])):
        cap = VideoCapture(str(current_ds[j]))
        frame_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))        
        klass = category
        out_prefix = klass 
        with mp_holistic.Holistic(
            min_detection_confidence = 0.5,
            min_tracking_confidence = 0.5
        ) as holistic:
            while cap.isOpened() and frame_length!=0:
                image: ndarray
                success, image = cap.read()
                if not success: continue
                image = flip(image, 1)

                image = cvtColor(image, COLOR_BGR2RGB)

                image.flags.writeable = False
                results = holistic.process(image)
                image = cvtColor(image, COLOR_RGB2BGR)
                image = im.draw_results(image, results)
                landmarks = me.extract_landmarks(results)
                QR.append(landmarks)
                frame_length-=1
        QR_vid = expand_dims(array(QR), 0)
        if QR_vid.shape == (1,50,1629):
            results = model.predict(QR_vid)
            results = array(results).mean(axis=0)
            predicted_kls = klasses[results.argmax(axis=0)]
            final_pred_list.append(predicted_kls)
            final_list.append(classes[i])
        else:
            print("WRONG SHAPE VIDEO"+ str(current_ds[j]) +str(j) + str(QR_vid.shape))
        print(j)
        QR = []
        if waitKey(5) & 0xFF == 27: break # 27 is escape!
    file1 = open("final_pred_list.txt","a")
    file1.write(str(final_pred_list))
    file1.write("\n")
    file1.close()
    final_final_pred_list.append(final_pred_list)
    final_pred_list =[]
    file2 = open("final_list.txt","a")
    file2.write(str(final_list))
    file2.write("\n")
    file2.close()
    final_final_list.append(final_list)
    final_list =[]
    cap.release()
print(final_final_pred_list)
print(final_final_list)
file1 = open("final_pred_list.txt","a")
file1.write(str(final_final_pred_list))
file1.close()
file2 = open("final_list.txt","a")
file2.write(str(final_final_list))
file2.close()

In [ ]:
print(final_final_pred_list)
print(final_final_list)

[['Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Work', 'Please', 'Phone', 'Please', 'Please', 'Phone', 'Please', 'Play', 'Phone', 'Phone', 'Please', 'Please', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Please', 'Phone', 'Work', 'Work', 'Phone', 'Play', 'Phone', 'Phone', 'Please', 'Work', 'Phone', 'Play', 'Please', 'Phone', 'Phone', 'Phone', 'Phone', 'Please', 'Work', 'Phone', 'Phone', 'Work', 'Phone', 'Work', 'Phone', 'Phone', 'Phone', 'Phone', 'Please', 'Phone', 'Phone', 'Phone', 'Phone', 'Work', 'Phone', 'Please', 'Please', 'Play', 'Phone', 'Please', 'Phone', 'Work', 'Phone', 'Phone', 'Phone', 'Phone', 'Please', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Please', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Please', 'Please', 'Please', 'Phone', 'Work'], ['Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Play', 'Your', 'Your', 'Play', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Y

In [ ]:
flat_list_pred = [item for sublist in final_final_pred_list for item in sublist]
flat_list_act = [item for sublist in final_final_list for item in sublist]

#Confusion matrix
y_actu = pd.Series(flat_list_act, name='Actual')
y_pred = pd.Series(flat_list_pred, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred,)
print(df_confusion)

In [ ]:
# tempo = [['Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie', 'Movie'],
# ['Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name', 'Name'],
# ['Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your', 'Your'],
# ['Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address'],
# ['Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play', 'Play'],
# ['Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work', 'Work'],
# ['Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please', 'Please'],
# ['Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone', 'Phone']]


In [ ]:
flat_list_act = [item for sublist in tempo for item in sublist]

#Confusion matrix
y_actu = pd.Series(flat_list_act, name='Actual')
y_pred = pd.Series(flat_list_pred, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred,)
print(df_confusion)

Predicted  Address  Name  Phone  Play  Please  Work  Your
Actual                                                   
Address         37     0     36     0       0     0    20
Movie            0     0     60     4      19    10     3
Name             0     0      0     4       0     2    84
Phone            1     0      7     5       0     0    77
Play             5     0      1     0       0     0    86
Please           0     0      0    17       0     0    77
Work             0     1     53    10      18    12     1
Your             2     0      0     0       0     1    86
